In [2]:
from datasets import load_dataset

ds = load_dataset("yoshitomo-matsubara/srsd-feynman_easy_dummy")

/home/mm/projects/ewolucyjne/ewolucyjne/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 30000/30000 [00:00<00:00, 1798021.21 examples/s]


In [20]:
print(ds['train'][8001])
print(ds['train'][0])
print(ds['train'][32001])

{'text': '7.619981169700622559e-01 1.818842291831970215e+00 8.440561314972055977e-27 2.070210048000000000e+09'}
{'text': '1.567521318793296814e-02 5.028885602951049805e-02 7.032883034485124934e-22 7.882885402068495750e-04'}
{'text': '3.148483825500160000e+14 1.329973754882812500e+03 -1.503542065620422363e-02 2.484383209680647575e-14 1.463717937323370766e+31 1.503295153379440308e-01'}


In [11]:
from datasets import load_dataset

ds_medium = load_dataset("yoshitomo-matsubara/srsd-feynman_medium")
print(ds_medium['train'][0])

ConnectionError: Couldn't reach 'yoshitomo-matsubara/srsd-feynman_medium' on the Hub (LocalEntryNotFoundError)

In [15]:

ds_medium = load_dataset("yoshitomo-matsubara/srsd-feynman_medium")

Generating test split: 100%|██████████| 40000/40000 [00:00<00:00, 2072182.20 examples/s]


In [17]:
print(ds_medium['train'][8001])
print(ds_medium['train'][0])

{'text': '2.660175152830161591e-23 -1.411560747911434488e+00 2.660175152102865016e-23'}
{'text': '-4.068378094392997468e+01 3.073896779351707101e+08 -3.521918534736374085e+01 2.149162971293337402e+12'}


In [21]:
print(ds_medium)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 320000
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 40000
    })
})


In [212]:
from dataclasses import dataclass
import sympy
import numpy as np
import pickle
from collections import deque

examples_size=8000
rng = np.random.default_rng()

# EXTRA_FUNC_DICT = {
#     'exp': make_function(np.exp, 'exp', 1),
#     'pow': make_function(np.power, 'pow', 2)
# }

STR2SYMPY = {
    'neg': lambda x: -x,
    'abs': sympy.Abs,
    'sqrt': sympy.sqrt,
    'exp': sympy.exp,
    'log': sympy.log,
    'sin': sympy.sin,
    'cos': sympy.cos,
    'tan': sympy.tan,
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x * y,
    'div': lambda x, y : x / y,
    'pow': lambda x, y : x ** y
}

binary_functions=np.array(['add','sub','mul','div','pow'])
unary_functions=np.array(['neg','sqrt','exp','log','sin','cos','tan'])
extra_constants=np.array(['2','3','4','5','pi'])

@dataclass
class ExpNode:
    kind: str
    num_children: int
    children: list
    parent: 'ExpNode' = None
    #

# @dataclass
# class Expression:
#     root: ExpNode
#     variables: np.ndarray
#     depth:int
def find_variables(root: ExpNode):
    if isinstance(root, np.ndarray):
        return root
    dir=rng.uniform(0,1)
    if dir<1/2 or root.num_children==1:
        return find_variables(root.children[0])
    elif dir>1/2 and root.num_children>1:
        return find_variables(root.children[1])
    
# def cut_and_regenerate(formula: ExpNode,depth,variables):
#     d=0
#     visiting=formula
#     while d<depth:
#         dir=rng.uniform(0,1)
#         if dir<1/2 or visiting.num_children==1:
#             visiting=visiting.children[0]
#         elif dir>1/2 and visiting.num_children>1:
#             visiting=visiting.children[1]
#         d+=1
#     if d==depth:
#         dir=rng.uniform(0,1)
#         if dir<1/2 or visiting.num_children==1:
#             visiting.children[0]=generate_Expression(variables)
#             visiting.children[0].parent=visiting
#         elif dir>1/2 and visiting.num_children>1:
#             visiting.children[1]=generate_Expression(variables)
#             visiting.children[1].parent=visiting

# def cut_and_shorten(formula: ExpNode,depth):
#     d=0
#     visiting=formula
#     while d<depth:
#         dir=rng.uniform(0,1)
#         if dir<1/2 or visiting.num_children==1:
#             visiting=visiting.children[0]
#         elif dir>1/2 and visiting.num_children>1:
#             visiting=visiting.children[1]
#         d+=1
#     if d==depth:
#         dir=rng.uniform(0,1)
#         if dir<1/2 or visiting.num_children==1:
#             visiting.children[0]=find_variables(visiting)
#         elif dir>1/2 and visiting.num_children>1:
#             visiting.children[1]=find_variables(visiting)

def visit(formula: ExpNode):
    tovisit=deque()
    tovisit.append((formula,None,0))
    count=0
    while tovisit:
        node,parent,j=tovisit.popleft()
        if not isinstance(node, np.ndarray):
            for i,child in enumerate(node.children):
                tovisit.append((child,node,i))
        count+=1
        if rng.integers(1,count+1)==count:
            item=node
            itemparent=parent
            itemid=j
    return item,itemparent,itemid

def cut_and_shorten(formula: ExpNode,depth):
    cutpoint,cutparent,cutid=visit(formula)
    if cutparent is not None:
        cutparent.children[cutid]=find_variables(cutpoint)
    return formula

def cut_and_regenerate(formula: ExpNode, variables):
    cutpoint,cutparent,cutid=visit(formula)
    if cutparent is not None:
        cutparent.children[cutid]=generate_Expression(variables)
        cutparent.children[cutid].parent=cutparent
    else:
        formula=generate_Expression(variables)
    return formula

def crossover(formula1: ExpNode,formula2: ExpNode):
    cxpoint1,parent1,id1=visit(formula1)
    cxpoint2,parent2,id2=visit(formula2)
    if parent1 is None:
        return cxpoint2
    else:
        cxpoint2.parent=parent1
        parent1.children[id1]=cxpoint2
        return parent1


def choose_function():
    fn_choice=rng.uniform(0,1)
    children=0
    if fn_choice>7/12:
        kind=rng.choice(binary_functions,1)
        children=2
    else:
        kind=rng.choice(unary_functions,1)
        children=1
    return kind[0],children

def generate_subtree(root,depth,max_depth,variables):
    if depth==max_depth:
        for n in range(root.num_children):
            var=rng.choice(variables,1)
            # if var[0]=='const':
            root.children.append(var)
    else:
        for n in range(root.num_children):
            fn,children=choose_function()
            child=ExpNode(fn,children,[],root)
            generate_subtree(child,depth+1,max_depth,variables)
            root.children.append(child)


def generate_Expression(variables: np.ndarray):
    depth=rng.integers(1,3)
    num_variables=rng.integers(1,variables.shape[0]+1)
    #check if replacement mayu be usefull (the same variable is used twice in different fragments of the expression)
    # we may choose just one variable then
    used_variables=rng.choice(variables,num_variables,replace=False)
    print(num_variables,used_variables)
    #using uniform generation
    d=0
    curr_fn,kids=choose_function()
    root=ExpNode(curr_fn,kids,[])
    generate_subtree(root,1,depth,np.concatenate([used_variables,extra_constants]))
    return root

        
# def parse_Expression(expr: ExpNode):
#     if type(expr)==np.ndarray:
#         return sympy.symbols(expr[0])
#     else:
#         print(expr.kind)
#         print((parse_Expression(expr.children[i]) for i in range(expr.num_children)))
#         return STR2SYMPY[expr.kind](parse_Expression(expr.children[i]) for i in range(expr.num_children))

def parse_Expression(expr: ExpNode):
    if isinstance(expr, np.ndarray):
        return sympy.symbols(expr[0])
    elif expr.kind in STR2SYMPY:
        # Parse children recursively
        parsed_children = [parse_Expression(child) for child in expr.children]
        # Apply the corresponding sympy function/operator
        return STR2SYMPY[expr.kind](*parsed_children)
    else:
        raise ValueError(f"Unsupported expression kind: {expr.kind}")


In [210]:
t=np.array(['x','y','z'],dtype=str)
res=generate_Expression(t)
print(res)

1 ['z']
ExpNode(kind=np.str_('neg'), num_children=1, children=[array(['2'], dtype='<U2')], parent=None)


In [217]:
print(visit(res))

(array(['2'], dtype='<U2'), ExpNode(kind=np.str_('neg'), num_children=1, children=[array(['2'], dtype='<U2')], parent=None), 0)


In [143]:
print(parse_Expression(res))

x**x


In [4]:
with open('srsd/medium_set/true_eq/feynman-i.8.14.pkl','rb') as eq:
    equaion=pickle.load(eq)
    print(equaion,type(equaion))

arr=np.loadtxt('srsd/medium_set/test/feynman-i.8.14.txt')
print(arr)
arr1=np.loadtxt('srsd/medium_set/test/feynman-i.34.8.txt')
print(arr1)

sqrt((x0 - x1)**2 + (x2 - x3)**2) <class 'sympy.core.power.Pow'>
[[ 2.00293401  0.76615789 -1.16301986 -2.29924894  1.67947364]
 [-0.1516021  -0.315528    4.28046425 -0.16175568  4.44524347]
 [ 7.22516759 -0.60686739  5.98877255 -2.48898992 11.54180351]
 ...
 [-0.38752721 -3.80981155 -4.00705763 -9.805081    6.73268929]
 [ 0.15153997  1.39060106 -0.13542526 -5.74073294  5.74062249]
 [-3.96895032 -3.69390316  2.53040955 -1.05597386  3.59691491]]
[[-7.80518190e-10 -1.96520295e+05 -1.75401610e+02  9.08206278e+09
  -2.96237144e-12]
 [ 2.97627266e-10  1.02646499e+06 -4.60135510e+02 -3.14479787e+09
   4.47002418e-11]
 [-6.97188338e-11  3.59188320e+05  5.93024628e+02 -4.42789509e+09
   3.35388160e-12]
 ...
 [ 9.81457051e-11  4.97903653e+06 -1.21105673e+01  6.71299609e+10
  -8.81586042e-14]
 [ 1.24657280e-10  2.96818907e+06 -1.39143860e+02  1.25997876e+10
  -4.08610979e-12]
 [-1.91575990e-11  7.69971207e+06  4.29408749e+01 -9.05855451e+10
   6.99242071e-14]]


In [208]:
print(rng.integers(1,2))
w=deque()
w.append('a')
w.append(17)
print(w)

1
deque(['a', 17])


In [8]:
print(type(t[0]))

<class 'numpy.str_'>
